In [1]:
import import_ipynb
import CA
import pprint
CAPubKeyC = []
reqFrame = {}
clientName = 'ECU1'
role = 4
ClientCert = []
clientPubKey = []
clientPriKey = []

importing Jupyter notebook from CA.ipynb


In [2]:
def CertificateRoutine():
    global ClientCert, clientPubKey, clientPriKey, CAPubKeyC, reqFrame, clientName, role
    import socket
    import pickle
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('127.0.0.7', 8080))
    global ClientCert
    if CAPubKeyC == []:
        reqFrame = {"req":1} # requesting for CA's PublicKey
        data = pickle.dumps(reqFrame)
        print("Requesting for CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        CAPubKeyC = pickle.loads(data)
        print("CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
    if ClientCert == []: #Generating Subject'S Public,Private Key pair
        if clientPubKey == []:
            clientPubKey , clientPriKey = CA.KeyGen()
        reqFrame = {"req" : 2, "subName" : clientName, "role":role, "pubKey" : clientPubKey} #Request for certificate
        data = pickle.dumps(reqFrame)
        print("Request for Certificate Generation \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        fromServer = pickle.loads(data)
#         print(fromServer)
        ClientCert = CA.certificate()
        ClientCert.dict2cert(fromServer)
        print("Generated Certificate \t:\t")
        pprint.pprint(ClientCert.transmitCertificate())
        print('\n')
    client.close()

In [3]:
import numpy as np

def RandomGenerator():
    random_bytes = str(np.random.bytes(10).hex())
    seed = random_bytes
#     for i in range(0,2):
#         random_bytes = str(np.random.bytes(1).hex())
#         seed = seed +' '+ random_bytes
    return seed.upper()

def Req(data):
    global ClientCert, clientPriKey
    if data["sFn"] == '08':
        print("From Tester : ")
        pprint.pprint(data)
        print("\n")
        from_client = {}
        from_client["Fn"] = '69'
        from_client["sFn"] = '08'
        from_client["Cnf"] = '02'
        print("To Tester : ")
        pprint.pprint(from_client)
        print("\n")
        return from_client
    elif data["sFn"] == '01' or data['sFn']=='02':
        print("--------------Unidirectional Authetication")
        print("\n\n")
        print("From Tester : ")
        pprint.pprint(data)
        print("\n")
        from_client = {}
        from_client["Fn"] = '69'
        from_client["sFn"] = data['sFn']
        from_client["Cnf"] = '11'
        t = RandomGenerator()
        from_client["CS"] = t
        if from_client["sFn"] == '02':
            temp_ccli = data["CCli"]
            from_client['ECer'] = ClientCert
            from_client['POOC'] = CA.sign(temp_ccli,clientPriKey)
        print("To Tester : ")
        pprint.pprint(from_client)
        print("\n")
        return from_client
    elif data["sFn"] == '03':
        print("From Tester : ")
        pprint.pprint(data)
        print("\n")
        from_client = {}
        from_client["Fn"] = '69'
        from_client["sFn"] = '03'
        from_client["Cnf"] = '12'
        print("To Tester : ")
        pprint.pprint(from_client)
        print("\n")
        return from_client
    return 0

In [4]:

def AuthenticationRequester():
    import socket
    import pickle
    serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    serv.bind(("127.0.0.1", 9090))
    serv.listen(5)
    a,b=0,0
    global RecCer, CAPubKeyC, ClientCert , clientPubKey, clientPriKey
    while a==0:
        b=0
        conn, addr = serv.accept()
        while b == 0:
            data = conn.recv(4096)
            if not data: break
            from_client = pickle.loads(data)
            if from_client["sFn"] == '08':
                tempData = Req(from_client)
                from_client = tempData
                data = pickle.dumps(from_client)
                conn.send(data)
                data = conn.recv(4096)
                from_client = pickle.loads(data)
            if from_client["sFn"] == '01' or from_client["sFn"] == '02':
                RecCert = from_client['Cert']
                if CA.verifyCertificate(RecCert,CAPubKeyC ):
                    print("--------------Certificate Verified\n")
                    tempData = Req(from_client)
                    from_client = tempData

                    t = from_client['CS']
                    data = pickle.dumps(from_client)
                    conn.send(data)
                    data = conn.recv(4096)
                    from_client = pickle.loads(data)
                    if from_client["sFn"] == '03':
                        if CA.verifySign(t,from_client["POO"],RecCert.subjectPublicKey):
                            print("--------------Certificate Verified with proof of Ownership\n")
                            tempData = Req(from_client)
                            from_client = tempData
                            data = pickle.dumps(from_client)
                            conn.send(data)
                            conn.close()
            conn.close()
            a,b=1,1

In [9]:
CertificateRoutine()

In [7]:
AuthenticationRequester()

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '8DDEDE24863B97A6EE3D',
 'Cert': <CA.certificate object at 0x0000019840AEE1C0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '96526251547CE99B9D81',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x00000198314D5D00>,
 'Fn': '69',
 'POOC': b'\xaf\x8e\xcb\xc5\xcc\x11\xb2dY\xe0\x02f`\x0e3\xdf`\x95\xb9$'
         b']\xd2\xb9\x91\xaf0\xb8\x85\xe5G}\xb0\xa7\xe1\xe7\xfa\x0ef\xb1\x87'
         b"]\x9a\x9c\x91\xd9\xed\x87\xe39>\xaeGdD\x8c\xd3F\xa8\x01\x1f/'\xd1_",
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'&:)\xc0\xc3\xf8\x085\xae\x87B\x00\xc3\xd7\xc4\xe7\xee\xfb\x8e\x80'
        b'CG\xac1\xcao\x1d\x9e\x8e\x87\xd0\xc6Q\xf3l8\xb4f:\x12\x

In [28]:
AuthenticationRequester()

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


Verification succeed
--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '4AC194C635569536BB00',
 'Cert': <CA.certificate object at 0x000001918CC192E0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': 'EA9DB1CCE316ECA89047',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x000001918CBD5A90>,
 'Fn': '69',
 'POOC': b'\x87s\x8e\x0b(\x12\x88\x9c\x1d\xbf\xca^>w\x0f\xef\xeb\x05\x8c\x13'
         b'\x91\xf7`HC\t5V\xe4\\\xfeY\x93"\x9f\n\x01\xd6]\x17\x00\xca\xbd^'
         b'\xd9\xef\xdf\x89u\xe7\x13\xbc\xd3\xc2\x92U\x12cL\xc7\x18\xa0\xdd\xef',
 'sFn': '02'}


Verification succeed
--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'\x0e\xdf\xce<?\xdb+L\xf1\xa8\x18_wc\xc7K\xa6\xc4\x7f\xa0+\x08x{'
        b'18\x8f\xa1\x05\xeeo\xd9\xdaw\xa6sM\x19xj\xbeN\x87\xc1\xfa9\xa9\x0c'
        b'\xf

In [11]:
for _ in range(0,100):
    AuthenticationRequester()

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '78C075048324CEA047C2',
 'Cert': <CA.certificate object at 0x000002026063B070>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': 'D029069EAC184FD0CF6B',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b"\x0f5\x99+\xae\x82\xc2\x88NF'Z\xb03\x87\xba \xb7\xdeeJ\xfdh!"
         b'8\xde\x14\nzR(*{\xd4\xab\xbb>\xaaC\x97\xaf#j\xc3c\x15-\x8b}\xb3\xbaf'
         b'\xce)%\x81@e\xcd\t\xbd\xf3\xb9\xf5',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'\x1dN\xa6\'y;\x9e\xe02G"\xd9%\x84\xa7\xd4\x89\x17s\x91\xceo\xf7\''
        b'\x9f\xf1\x97\xb9M\xa8mwq\xaa\xf9\x08\x9d\x8b\xee\xb9y$3t\x13\x9aS\x1c'
        b'.-

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '796431E8023DC80F3B08',
 'Cert': <CA.certificate object at 0x00000202510032E0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '4C4E0C8E4D7B89D57AE8',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b'd\xdd"\xdb\xd6\xa8\x92\x14\x0bF\xb8\xcf\x03,\xce\xfe\xe4\xc0oj'
         b'\xf4\xf0\xba\x8a6n\x8et\xd8[z\xfeJ\x85\xcbg\x8c\x9do\x86'
         b'\xa5\xa9\xa4\x1c;\x92j!\xba\x89\x91\x15y\x84i\xaa\x95ev\x80'
         b'\xbe\xf3,\xb8',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'\x0e\x80\x8f\xfe)r\xeds\xe3(&\xa7u\xd9C\x0f\x8fC\xfdK`\xac\xdb\xc0'
        b'W\xb9:\xcc\xa3\xb0\x91\x8be#^\xbd\x03p\xe

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': 'F2E6C3CD7BE7A38FFA41',
 'Cert': <CA.certificate object at 0x0000020251294580>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': 'AAD61D2DF0BF4D815D1E',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b'x\xa7!\t\xb0\x1bJt\x8f\x1c5G\x9c[e\x8dm\xfd\xcc>-H\xf3\x1a'
         b'<\xb0\xf6\xc5\xab\xd2~\x99\x99jl\x854\xed\x8f\x07B\x8b\x90\x9c'
         b'\x8e\xf3\x12\xaf\xc5\x08(l\xbe\xa8v\xd2\x82\x8fV\x06\x84\xec\xa7\x80',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'\x7f\xe3\xa6v\x93\x930\xb8\x98\xa0\x03\x13{{0\xb0D\xfd\xec\xf9'
        b'u\xf3\xdf+\xb3\x8b\x06\xf6\xe2\x03\xea\xa9T\xcc\xee\xa9Ti\x9

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': 'CF243680AE120343EC90',
 'Cert': <CA.certificate object at 0x000002025128FD00>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': 'CDEA51227274F271F9FA',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b"\x0c1\xd1\xaf\xb6\xe2b'\xb5\x06\xdd7\xdbn\xa2\x9c\xc5\x17_\r"
         b'nR\x02\xb6\xf0A\xe0dE\xe0\xca\xa3\xec3\xf3\xb5\xc6\xf47\xc7'
         b'\xdd\xe3\xb63\x8aP\xa3X\xb7p\xf1\xb4\x8d\xa9\x08\xb5\xde)}{'
         b'fd\xed\xe5',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'\x83toL\x1c\xb7\xcf\xdf\xf4C\x07\x83\x90U\x0f\xb0\xb0\x92\xf1\x99'
        b"t,\xa5W\xee\xa7u\xd1I\xd4UuO\x8e'\xf1\x8a\xd

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': 'EA713EAC7CCD2C8EC8ED',
 'Cert': <CA.certificate object at 0x000002026063B100>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '4EAC16B15434AD1856D8',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b'{\x06\x13\xe3?\xd1\xd1_\xb5{\x91\xa1\xc1f\xd9\xa4\x87\\\xa6\xce'
         b'n\xb5]2\x08\x9e\x9c+\xc6\x9a\x16\xc5E\xdc\xe5r\xd7f"z\xcb\xc7*>'
         b"\x17\xc6\xb0\x0f\x7f\xbf\xe6\xf6ev\xdd\x84\xa9'\xff\xb0\x17\x0e\xbcH",
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b',\xafH\xce\xe5\xfe\x1d\xb8\xd16r\n|\x93\xaf\xde\xf0\x81\x98d'
        b'\xee#\x0e8\ri\xbf\xcfhU\xb7\xcc\xfa\xd5h\xa5\xb6\x86.['


...

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '52C3FF0EDA06E31B4FC8',
 'Cert': <CA.certificate object at 0x000002025126BDF0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '192D05790F1B6DFC064C',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b"\x11\x00N1o\x89\x83\xecv,gk\x8a\r\xca\xf7\xd5\xc2\x15\x0c\xc3\xaa'<"
         b'$\x84\xd4}\x11\xea\x85\x0f\x14i\xd8\xa1l\x87\xfa|Q\xa0r>'
         b'U\xbc\xcd\xd3\x17\xa1?\xe4\x14\xab\xe7\xb7,\xf5\xdf8\x8e\x06\xdc\xe6',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'\x88\xd7\xf4b\x03*\x9d\xc5\xb2u\xc74\x9a\xb5\x08y,uM\xdd\xc4\xecs\xa0'
        b'x\xa9\xfd\xcd\x0bw\x9d\xffe\x8c\xff\xbcS~=\x81\x89

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '013BED8F167E4E71152F',
 'Cert': <CA.certificate object at 0x000002025128F040>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '7A76DB2ECB021BF07DBA',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b'\x9ea(\xb0\xd3\xb5\x82\xd2\x10\x96HLJ1\xf93\xf7\xd1\xa5\x8a'
         b'\xbb\xe2\xf4\x0b\n\x10a9\xcc\xda\x16O\xb9\xf4\x8322\xab\xe04X_L\xf6'
         b'\x1d\x1d\x0c>\xce\xabO\xcbX\x0e64\xad\xab\xfb\xe1o\x98\x84\x9c',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'o\x9f\xd9\xe1xbBD\xa1\x00y\x05\x1a\xf8\x8dU\xc6\x81\x1a\xbbHR\xa2V'
        b'\xb5\x0e\xf9\x93|\xa4B\xd2\x9aj\x85YS\xde\x118\xc6o\xb7\

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '8778B68C39E76E9196EB',
 'Cert': <CA.certificate object at 0x000002025125AA60>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '94B995ABE526F21CF5D1',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b'\x16\x18\xab\x87\xfcH\xc7G?2\xed,"\xe2\x19R\xcd7\xbc\xe5S_\x8f\x88'
         b"\xd4lb'\xc21\xbf\x83s\xe4\x9f\xa4\xb9\xdf\x9eI\x9d\xddt$"
         b'\x0c\xdd\x9c\xbe\xe5\x0c\xb7\xb5\xc2lUp\xee\x9b\xd5\xbd'
         b'\xb1\xac\x99\n',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b'2\xd3UP\xae\x00r\xc8!\x1f\xa2\xb0\n\xe0Gb~/rq\x97B+Eo\x1d\x9b\xf1'
        b'gdR\xb6\x93\x98\x8e\xda\x18\x15\xd8/c\xe9

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': '7048618FF2BB94CE169B',
 'Cert': <CA.certificate object at 0x000002025126BDF0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': '5690584EC6BA106D6AA2',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b'w\xf5<\xbc\xceN~/\\\xf1\x9fa\xc7-\x05\xe8\x80"\xf0\x17?\xe6\x91\xc9'
         b'\xd9*\xdb\xd0b\x04\xfb\xf1\x1d1\x85\xca\xaf+\x9b.M\x8a\x93\x08'
         b'\x1d4/%\x94R)h8\xad~\xbb@\x82\xc9\xabL\n\xb4F',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b"2\xb9V\xd3\xc7\x8f\xb9\xa45N.\r\xc5r:\x91\xeb\xf8:\xa8\x15>'\xd1"
        b'F\xae\x1f\x04R\xb2\xf2\xd1%}6\xf8]\xd9\xe7K\x06L\xed1Vy|<'
        b'\xc

From Tester : 
{'Fn': '29', 'sFn': '08'}


To Tester : 
{'Cnf': '02', 'Fn': '69', 'sFn': '08'}


--------------Verification succeed

--------------Certificate Verified

--------------Unidirectional Authetication



From Tester : 
{'CC': '00',
 'CCli': 'F69039F22A1187FA0644',
 'Cert': <CA.certificate object at 0x000002025128F9A0>,
 'Fn': '29',
 'sFn': '02'}


To Tester : 
{'CS': 'A77788716496D26563D7',
 'Cnf': '11',
 'ECer': <CA.certificate object at 0x0000020251262B20>,
 'Fn': '69',
 'POOC': b'\xa0\xe3\xd1\x84C\xdc\xab\x7f\x1d!\xe1p\x86\xdb\xcb\x1cY\xc85\xed'
         b'5\xfd\x8d\xdc\xf36co\x91;\xb6\x9aZ-0p\xf3\x07>\xb6\xdbj\xb8\x19'
         b'O\x95\x0b\t[\xf6z\xd9\x8c3\x80{\xf4\x83\xf4\x1e\xe9.\xbes',
 'sFn': '02'}


--------------Verification succeed

--------------Certificate Verified with proof of Ownership

From Tester : 
{'Fn': '29',
 'POO': b"Z\xf3\xe6\xbc\xe0'\xf8\x94T\xd1\xfa^\xfa\xbe@^\r\xa2\x0cs\xa2\xcd-|"
        b'\x05\xac\xa6F\xfb\x8b\x8a@\xc3\x9ce\x04\xb4\x84\xaf\x8d*[\

In [ ]:
AuthenticationRequester()